# 04: experiments

this notebook runs systematic experiments and compares results from the model registry.


In [1]:
import sys
sys.path.append('../src')

import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from src.config import Config
from src.datasets import get_cifar10_loaders
from src.models import create_model
from src.trainer import Trainer
from src.utils import set_seed, calculate_model_size
from src.model_registry import registry

set_seed(42)
config = Config()
print(f"device: {config.device}")


device: cuda


In [2]:
# load data
train_loader, val_loader, test_loader = get_cifar10_loaders(config)
print(f"data loaded: {len(train_loader.dataset)} train, {len(val_loader.dataset)} val, {len(test_loader.dataset)} test")


data loaded: 40000 train, 10000 val, 10000 test


In [3]:
# analyze existing models
models = registry.list_models()
results = []

for model_id, info in models.items():
    model_name = info['model_name']
    history = info['history']
    parameters = info['parameters']
    
    # get best validation accuracy
    best_val_acc = max(history['val_acc'])
    final_train_acc = history['train_acc'][-1]
    final_val_acc = history['val_acc'][-1]
    
    results.append({
        'model_id': model_id,
        'model_name': model_name,
        'parameters': parameters,
        'best_val_acc': best_val_acc,
        'final_train_acc': final_train_acc,
        'final_val_acc': final_val_acc,
        'epochs': len(history['train_acc'])
    })

df = pd.DataFrame(results)
print("model comparison:")
print(df.to_string(index=False))


model comparison:
                   model_id  model_name  parameters  best_val_acc  final_train_acc  final_val_acc  epochs
  SimpleCNN_20251024_171753   SimpleCNN     1147914         83.50          82.7875          83.50      30
ImprovedCNN_20251024_172317 ImprovedCNN     1186826         84.21          86.1900          83.42      30
  SimpleCNN_20251024_174825   SimpleCNN     1147914         83.50          82.7875          83.50      30
ImprovedCNN_20251024_175420 ImprovedCNN     1186826         84.21          86.1900          83.42      30
  SimpleCNN_20251024_180951   SimpleCNN     1147914         81.58          78.7125          81.58      20
  SimpleCNN_20251024_181037   SimpleCNN     1147914         80.59          78.6425          80.59      20


In [5]:
#running new experiments with different hyperparameters
print("running new experiment with different learning rate...")

#creating new config with different lr
new_config = Config()
new_config.lr = 0.01
new_config.epochs = 20
print(f"new config: lr={new_config.lr}, epochs={new_config.epochs}")

#training simple cnn with new config
model = create_model('simple_cnn', new_config).to(new_config.device)
trainer = Trainer(model, new_config)

print("training new model...")
history = trainer.fit(train_loader, val_loader)

print(f"new model best val acc: {max(history['val_acc']):.2f}%")
print(f"new model final train acc: {history['train_acc'][-1]:.2f}%")


running new experiment with different learning rate...
new config: lr=0.01, epochs=20
training new model...
Training on cuda
Model parameters: 1,147,914

Epoch 1/20
--------------------------------------------------


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 184.97it/s]


Train Loss: 1.6341, Train Acc: 39.21%
Val Loss: 1.2767, Val Acc: 52.89%
New best validation accuracy: 52.89%

Epoch 2/20
--------------------------------------------------


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 212.82it/s]


Train Loss: 1.3216, Train Acc: 51.85%
Val Loss: 1.0962, Val Acc: 59.81%
New best validation accuracy: 59.81%

Epoch 3/20
--------------------------------------------------


...

Train Loss: 0.6393, Train Acc: 77.58%
Val Loss: 0.6546, Val Acc: 77.60%

Epoch 19/20
--------------------------------------------------


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 217.25it/s]


Train Loss: 0.6321, Train Acc: 78.10%
Val Loss: 0.5808, Val Acc: 79.52%
New best validation accuracy: 79.52%

Epoch 20/20
--------------------------------------------------


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 228.61it/s]

Train Loss: 0.6139, Train Acc: 78.64%
Val Loss: 0.5209, Val Acc: 82.04%
New best validation accuracy: 82.04%

Training completed in 44.25s
Best validation accuracy: 82.04%
model registered with id: SimpleCNN_20251024_181723
new model best val acc: 82.04%
new model final train acc: 78.64%


In [6]:
# compare all models
print("final model comparison:")
models = registry.list_models()
for model_id, info in models.items():
    print(f"{info['model_name']}: {max(info['history']['val_acc']):.2f}% val acc, {info['parameters']:,} params")

final model comparison:
SimpleCNN: 83.50% val acc, 1,147,914 params
ImprovedCNN: 84.21% val acc, 1,186,826 params
SimpleCNN: 83.50% val acc, 1,147,914 params
ImprovedCNN: 84.21% val acc, 1,186,826 params
SimpleCNN: 81.58% val acc, 1,147,914 params
SimpleCNN: 80.59% val acc, 1,147,914 params
SimpleCNN: 80.50% val acc, 1,147,914 params
SimpleCNN: 82.04% val acc, 1,147,914 params
